In [6]:
#Librerias a utilizar---------------------------------------------------------------------------------------------------------
import pandas as pd
import os
import re
import numpy as np
from scipy.signal import find_peaks
from scipy.signal import butter, filtfilt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler


#-----------------------------------------------------------------------------------------------------------------------------

#FUNCIONES IMPLEMENTADAS------------------------------------------------------------------------------------------------------

#Lectura de datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def separar_datos(linea):
  """
  Separa una línea del archivo de texto en tres valores de columna.

  Argumentos:
    linea: Cadena de texto que representa una línea del archivo.

  Devuelve:
    Tupla con los valores para las tres columnas o `None` si no hay coincidencia.
  """
  match = re.match(r'(\d+)\t0\t(\d+)\t(\d+)', linea)
  if match:
    return (match.group(1), match.group(2), match.group(3))
  else:
    return None

def leer_datos_txt(ruta_archivo):
  """
  Lee el contenido de un archivo de texto y lo devuelve como un DataFrame de Pandas con tres columnas.

  Argumentos:
    ruta_archivo: Ruta completa del archivo a leer.

  Devuelve:
    Un DataFrame de Pandas con tres columnas o `None` si hay un error.
  """
  if not os.path.exists(ruta_archivo):
    print(f"Error: El archivo '{ruta_archivo}' no existe.")
    return None

  datos = []
  with open(ruta_archivo, 'r') as f:
    for linea in f:
      valores_columna = separar_datos(linea)
      if valores_columna:
        datos.append(valores_columna)

  df = pd.DataFrame(datos, columns=['# de muestra', 'BVP', 'ECG'])
  return df

def leer_datos_csv(ruta_archivo):
  """
  Lee el contenido de un archivo CSV y lo devuelve como un DataFrame de Pandas.

  Argumentos:
    ruta_archivo: Ruta completa del archivo a leer.

  Devuelve:
    Un DataFrame de Pandas con una columna adicional de numeración.
  """
  if not os.path.exists(ruta_archivo):
    print(f"Error: El archivo '{ruta_archivo}' no existe.")
    return None

  df = pd.read_csv(ruta_archivo, header=None)
  df.columns = ['BVP', 'ECG']
  df.insert(0, '# de muestra', range(1, len(df) + 1))
  return df

def lectura_datos(ruta_archivo):
  """
  Lee el contenido de un archivo de texto o CSV y lo devuelve como un DataFrame de Pandas con tres columnas.
  Luego escala los valores de la columna '# de muestra'.

  Argumentos:
    ruta_archivo: Ruta completa del archivo a leer.
    valor_a_escalar: Valor para escalar la columna '# de muestra'.

  Devuelve:
    DataFrame de Pandas con la columna '# de muestra' escalada.
  """
  ext = os.path.splitext(ruta_archivo)[1].lower()
  if ext == '.txt':
    df = leer_datos_txt(ruta_archivo)
  elif ext == '.csv':
    df = leer_datos_csv(ruta_archivo)
  else:
    print(f"Error: El archivo '{ruta_archivo}' tiene una extensión no soportada.")
    return None

  return df

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Convertir a enteros los datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def convertir_columnas_a_enteros(df):
  # Convertir las columnas a números enteros
  df['# de muestra'] = pd.to_numeric(df['# de muestra'])
  df['ECG'] = pd.to_numeric(df['ECG'])
  df['BVP'] = pd.to_numeric(df['BVP'])
  return df

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Convertir muestras a tiempo en segundos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def convertir_muestras_a_tiempo(df):
  # Frecuencia de muestreo
  sample_rate=1000
  
  # Crear el eje de tiempo en segundos para las columnas BVP y ECG
  time = np.arange(len(df['# de muestra'])) / sample_rate

  # Agregar el eje de tiempo al DataFrame
  df['Time (s)'] = time

  # Tiempo maximo que se le puede dar al rango de graficacion
  max_value_time = df['Time (s)'].max()
  
  return df, max_value_time

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Normalizacion de datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def normalizar_datos(df):
  # Normalización Min-Max
  scaler_min_max = MinMaxScaler()
  df[['BVP_minmax', 'ECG_minmax']] = scaler_min_max.fit_transform(df[['BVP', 'ECG']])

  #Crear un nuevo dataframe con los datos BVP y ECG filtrados-----------------------
  df_filtered = df[['BVP_minmax', 'ECG_minmax', 'Time (s)']].copy()
  return df_filtered

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#filtrado de datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Filtrado PAM-TOMPKINS------------------------------------------
# Filtro paso alto para eliminar la línea de base
# Filtro de derivación
# Filtro paso bajo
# Detectar picos R
#----------------------------------------------------------------

# Filtro Paso alto-----------------------------------------------
def butter_highpass(cutoff, fs, order):
  nyq = 0.5 * fs
  normal_cutoff = cutoff / nyq
  b, a = butter(order, normal_cutoff, btype='high', analog=False)
  return b, a

def highpass_filter(data, cutoff, fs, order):
  b, a = butter_highpass(cutoff, fs, order=order)
  y = filtfilt(b, a, data)
  return y

# Filtro paso bajo-----------------------------------------------
def butter_lowpass(cutoff, fs, order):
  nyq = 0.5 * fs
  normal_cutoff = cutoff / nyq
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  return b, a

def lowpass_filter(data, cutoff, fs, order):
  b, a = butter_lowpass(cutoff, fs, order=order)
  y = filtfilt(b, a, data)
  return y

# Filtro de derivación -------------------------------------------
def derivative_filter(data, fs):
  diff = np.diff(data)
  diff = np.append(diff, 0)  # Añadir un 0 al final para mantener el mismo tamaño
  return diff * fs

def senial_ECG_pam_tompkins(df, order, cutoff_high, cutoff_low):

  fs = 1 / (df.iloc[1, 2] - df.iloc[0, 2])  # frecuencia de muestreo

  # Aplicar el filtro pasa alto a las señales ECG
  df['ECG_filtered'] = highpass_filter(df['ECG_minmax'], cutoff_high, fs, order)

  # Aplicar el filtro de derivación a las señales filtradas
  df['ECG_derivada'] = derivative_filter(df['ECG_filtered'], fs)

  # Aplicar el filtro pasa bajo a la señal derivada
  df['ECG_filtered_minmax'] = lowpass_filter(df['ECG_derivada'], cutoff_low, fs, order)

  # Crear un nuevo dataframe con los datos ECG filtrados y derivados
  df_filtered = df[['ECG_filtered_minmax', 'Time (s)']].copy()

  return df_filtered

def senial_BVP_paso_bajo(df, order, cutoff):
  
  fs = 1 / (df.iloc[1, 2] - df.iloc[0, 2])  # frecuencia de muestreo

  # Aplicar el filtro a las señales BVP
  df['BVP_filtered_minmax'] = lowpass_filter(df['BVP_minmax'], cutoff, fs, order)

  #Crear un nuevo dataframe con los datos BVP filtrados-----------------------
  df_filtered = df[['BVP_filtered_minmax', 'Time (s)']].copy()

  return df_filtered

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Señal ECG y BVP Picos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def RR_interval(df_subset, tiempo_ini, tiempo_fin, prominence_ECG,height_ECG,distance_ECG):
  # Seleccionar las muestras que se deseen graficar (entre un tiempo inicial y tiempo final en '# de muestra')
  df = df_subset[(df_subset['Time (s)'] >= tiempo_ini) & (df_subset['Time (s)'] <= tiempo_fin)]

  # Encontrar los picos en el subconjunto de datos
  peaks_ecg_subset, _ = find_peaks(df['ECG_filtered_minmax'], prominence=prominence_ECG, height=np.max(df['ECG_filtered_minmax']) * height_ECG, distance=distance_ECG)

  # Crear la figura con Plotly
  fig = go.Figure()

  # Graficar las muestras de ECG en el rango seleccionado
  fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['ECG_filtered_minmax'], mode='lines', name='Tiempo (s) vs mV', line=dict(width=0.5)))
  fig.add_trace(go.Scatter(x=df['Time (s)'].iloc[peaks_ecg_subset], y=df['ECG_filtered_minmax'].iloc[peaks_ecg_subset], mode='markers', name='Picos más altos', marker=dict(size=5)))

  # Calcular las distancias entre todos los picos
  rr_intervals = []
  if len(peaks_ecg_subset) > 1:
    for i in range(1, len(peaks_ecg_subset)):
      x_prev = df.loc[df.index[peaks_ecg_subset[i-1]], 'Time (s)']
      x_curr = df.loc[df.index[peaks_ecg_subset[i]], 'Time (s)']
      # Las distancias entre los distintos picos se restan en segundos, pero se almacenan en milisegundos en el vector
      rr_interval = (x_curr - x_prev) * 1000
      rr_intervals.append(rr_interval)

  fig.update_layout(
    title='Gráfico ECG (Intervalo de tiempo definido)',
    xaxis_title='Tiempo (s)',
    yaxis_title='Voltaje (mV)',
    legend=dict(orientation='h', y=-0.2),
    width=1000,  # Ajustar el ancho de la figura
    height=600,  # Ajustar el alto de la figura
    margin=dict(l=0, r=0, t=40, b=40)
  )

  fig.show()
  # fig.show(renderer="browser")

  # Retornar la cantidad de picos y las distancias entre los picos
  return len(peaks_ecg_subset), rr_intervals, peaks_ecg_subset

def IBI_BVP_interval(df_subset, tiempo_ini, tiempo_fin,prominence_BVP,height_BVP,distance_BVP):
  # Seleccionar las muestras que se deseen graficar (entre tiempo inicial y tiempo final en '# de muestra')
  df = df_subset[(df_subset['Time (s)'] >= tiempo_ini) & (df_subset['Time (s)'] <= tiempo_fin)]

  # Encontrar los picos en el subconjunto de datos
  peaks_bvp_subset, _ = find_peaks(df['BVP_filtered_minmax'], prominence=prominence_BVP, height=np.max(df['BVP_filtered_minmax']) * height_BVP, distance=distance_BVP)

  # Crear la figura con Plotly
  fig = go.Figure()

  # Graficar las muestras de BVP en el rango seleccionado
  fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['BVP_filtered_minmax'], mode='lines', name='Tiempo (s) vs BVP', line=dict(width=0.5)))
  fig.add_trace(go.Scatter(x=df['Time (s)'].iloc[peaks_bvp_subset], y=df['BVP_filtered_minmax'].iloc[peaks_bvp_subset], mode='markers', name='Picos más altos', marker=dict(size=5)))

  # Calcular y graficar las distancias entre todos los picos
  rr_intervals = []
  if len(peaks_bvp_subset) > 1:
    for i in range(1, len(peaks_bvp_subset)):
      x_prev = df.loc[df.index[peaks_bvp_subset[i-1]], 'Time (s)']
      x_curr = df.loc[df.index[peaks_bvp_subset[i]], 'Time (s)']
      # Las distancias entre los distintos picos se restan en segundos, pero se almacenan en milisegundos en el vector
      rr_interval = (x_curr - x_prev) * 1000
      rr_intervals.append(rr_interval)

  fig.update_layout(
    title='Gráfico BVP (Intervalo de tiempo)',
    xaxis_title='Tiempo (s)',
    yaxis_title='BVP',
    legend=dict(orientation='h', y=-0.2),
    width=1000,  # Ajustar el ancho de la figura
    height=600,  # Ajustar el alto de la figura
    margin=dict(l=0, r=0, t=40, b=40)
  )

  fig.show()

  # Retornar la cantidad de picos y las distancias entre los picos
  return len(peaks_bvp_subset), rr_intervals, peaks_bvp_subset

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Tratamiento de datos atipicos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def tratamiento_datos_atipicos_RR(df_datos_atipicos):
  # Convertir a un DataFrame para facilitar el manejo
  df = pd.DataFrame(df_datos_atipicos, columns=['Intervalo_RR'])

  # Calcular el IQR
  Q1 = df['Intervalo_RR'].quantile(0.25)  # Primer cuartil
  Q3 = df['Intervalo_RR'].quantile(0.75)  # Tercer cuartil
  IQR = Q3 - Q1  # Rango intercuartílico

  # Definir límites para detectar outliers
  limite_inferior = Q1 - 2 * IQR
  limite_superior = Q3 + 2 * IQR

  # Identificar outliers
  outliers = df[(df['Intervalo_RR'] < limite_inferior) | (df['Intervalo_RR'] > limite_superior)]

  print("Outliers detectados:")
  print(outliers)

  # Calcular la mediana para reemplazar los outliers
  mediana = df['Intervalo_RR'].median()

  # Reemplazar los outliers con la mediana
  df['Intervalo_RR'] = np.where((df['Intervalo_RR'] < limite_inferior) | (df['Intervalo_RR'] > limite_superior), mediana, df['Intervalo_RR'])

  # print("\nDataFrame después de tratar los outliers:")
  # print(df)
  return df

def tratamiento_datos_atipicos_IBI(df_datos_atipicos):
    # Convertir a un DataFrame para facilitar el manejo
    df = pd.DataFrame(df_datos_atipicos, columns=['Intervalo_IBI'])

    # Calcular el IQR
    Q1 = df['Intervalo_IBI'].quantile(0.25)  # Primer cuartil
    Q3 = df['Intervalo_IBI'].quantile(0.75)  # Tercer cuartil
    IQR = Q3 - Q1  # Rango intercuartílico

    # Definir límites para detectar outliers
    limite_inferior = Q1 - 2.5 * IQR
    limite_superior = Q3 + 2.5 * IQR

    # Identificar outliers
    outliers = df[(df['Intervalo_IBI'] < limite_inferior) | (df['Intervalo_IBI'] > limite_superior)]

    print("Outliers detectados:")
    print(outliers)

    # Calcular la mediana para reemplazar los outliers
    mediana = df['Intervalo_IBI'].median()

    # Reemplazar los outliers con la mediana
    df['Intervalo_IBI'] = np.where((df['Intervalo_IBI'] < limite_inferior) | (df['Intervalo_IBI'] > limite_superior), mediana, df['Intervalo_IBI'])

    # print("\nDataFrame después de tratar los outliers:")
    # print(df)
    return df


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#Calculo de parametros RR, IBI %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def parametros_RR(df_procesada_1, tiempo_inicial, tiempo_final, prominence, height, distance):

  tiempo_intervalo = tiempo_final - tiempo_inicial

  # Picos, cálculo de parámetros y graficación de picos
  cantidad_picos, distancias, data_ecg_peaks = RR_interval(df_procesada_1, tiempo_inicial, tiempo_final, prominence, height, distance)
  
  # Convertir distancias a un DataFrame si no lo es
  distancias = pd.DataFrame(distancias, columns=['Intervalo_RR'])
  
  # Tratar datos atípicos
  distancias = tratamiento_datos_atipicos_RR(distancias)
  
  print(f"Cantidad de picos: {cantidad_picos}")
  print(tiempo_intervalo)

  # Calculo Heart rate (se transforma de segundos a minutos)
  heart_rate = cantidad_picos / (tiempo_intervalo * (1 / 60))
  print(f"HR (picos/minutos): {heart_rate}")

  # Valor promedio RR
  RR_prom = distancias['Intervalo_RR'].mean()
  print(f"RR promedio (milisegundos): {RR_prom}")

  # SDNN
  RR_sdnn = distancias['Intervalo_RR'].std()
  print(f"RR sdnn (milisegundos): {RR_sdnn}")

  # RMSDD
  RR_rmssd = np.sqrt(np.mean(np.square(np.diff(distancias['Intervalo_RR']))))
  print(f"RR rmssd (milisegundos): {RR_rmssd}")

  # pNN50%
  # Calcular las diferencias entre elementos consecutivos
  RR_differences_ms = np.diff(distancias['Intervalo_RR'])
  # Obtener los valores absolutos
  RR_differences_absolute_values = np.abs(RR_differences_ms)
  # Contar la cantidad de valores mayores a 50 milisegundos
  RR_mayores_50 = np.sum(RR_differences_absolute_values > 50)
  # Contar el número de elementos en el vector
  Total_number_differences = len(RR_differences_absolute_values)
  # Calculo del PNN50%
  RR_pnn50 = (RR_mayores_50 / Total_number_differences) * 100
  
  print(f"Cantidad de valores mayores a 50 ms: {RR_mayores_50}")
  print(f"Cantidad de valores diferenciales: {Total_number_differences}")
  print(f"PNN50% (%): {RR_pnn50}")

  # SDSD
  RR_sdsd = np.std(RR_differences_absolute_values)
  print(f"RR sdsd (milisegundos): {RR_sdsd}")

  return heart_rate, RR_sdnn, RR_rmssd, RR_sdsd, RR_pnn50, data_ecg_peaks


def parametros_IBI(df_procesada_1, tiempo_inicial_2, tiempo_final_2, prominence_BVP, height_BVP, distance_BVP):

  time_2 = tiempo_final_2 - tiempo_inicial_2

  # Picos, cálculo de parámetros y graficación de picos
  cantidad_picos_2, distancias_2, data_BVP_peaks = IBI_BVP_interval(df_procesada_1, tiempo_inicial_2, tiempo_final_2, prominence_BVP, height_BVP, distance_BVP)

  # Convertir distancias a un DataFrame si no lo es
  distancias_2 = pd.DataFrame(distancias_2, columns=['Intervalo_IBI'])

  # Tratar datos atípicos
  distancias_2 = tratamiento_datos_atipicos_IBI(distancias_2)

  print(f"Cantidad de picos: {cantidad_picos_2}")
  print(time_2)

  # Calculo Heart rate (se transforma de segundos a minutos)
  heart_rate_2 = cantidad_picos_2 / (time_2 * (1 / 60))
  print(f"HR (picos/minutos): {heart_rate_2}")

  # Valor promedio IBI
  IBI_prom = distancias_2['Intervalo_IBI'].mean()
  print(f"IBI promedio (milisegundos): {IBI_prom}")

  # SDNN
  IBI_sdnn = distancias_2['Intervalo_IBI'].std()
  print(f"IBI sdnn (milisegundos): {IBI_sdnn}")

  # RMSDD
  IBI_rmssd = np.sqrt(np.mean(np.square(np.diff(distancias_2['Intervalo_IBI']))))
  print(f"IBI rmssd (milisegundos): {IBI_rmssd}")

  # pNN50%
  # Calcular las diferencias entre elementos consecutivos
  IBI_differences_ms = np.diff(distancias_2['Intervalo_IBI'])
  # Obtener los valores absolutos
  IBI_differences_absolute_values = np.abs(IBI_differences_ms)
  # Contar la cantidad de valores mayores a 50 milisegundos
  IBI_mayores_50 = np.sum(IBI_differences_absolute_values > 50)
  # Contar el número de elementos en el vector
  Total_number_differences_IBI = len(IBI_differences_absolute_values)
  # Calculo del PNN50%
  IBI_pnn50 = (IBI_mayores_50 / Total_number_differences_IBI) * 100

  print(f"Cantidad de valores mayores a 50 ms: {IBI_mayores_50}")
  print(f"Cantidad de valores diferenciales: {Total_number_differences_IBI}")
  print(f"PNN50% (%): {IBI_pnn50}")

  # SDSD
  IBI_sdsd = np.std(IBI_differences_absolute_values)
  print(f"IBI sdsd (milisegundos): {IBI_sdsd}")

  return heart_rate_2, IBI_sdnn, IBI_rmssd, IBI_sdsd, IBI_pnn50, data_BVP_peaks

def RR_Correlated_intervals(peaks_ecg_ms, peaks_bvp_ms):
  # Crear una lista para almacenar los tiempos de ECG y BVP correlacionados
  correlated_data = []

  # Iterar sobre los picos detectados en ECG
  for i in range(len(peaks_ecg_ms)):
    ecg_time = peaks_ecg_ms[i]

    # Encontrar el punto en BVP que cumple con la condición
    found_bvp_value = False
    for j in range(len(peaks_bvp_ms)):
      bvp_time = peaks_bvp_ms[j]

      # Verificar si el punto de BVP está dentro del rango del punto de ECG actual y el siguiente punto de ECG
      if ecg_time <= bvp_time <= (ecg_time if i == len(peaks_ecg_ms) - 1 else peaks_ecg_ms[i + 1]):
        # Agregar tiempos al DataFrame de datos correlacionados
        correlated_data.append({'ECG_PAT': ecg_time, 'BVP_PAT': bvp_time})
        found_bvp_value = True
        break

    # Si no se encuentra un punto de BVP que cumpla con la condición, asignar un valor de cero
    if not found_bvp_value:
      correlated_data.append({'ECG_PAT': ecg_time, 'BVP_PAT': 0})

  # Convertir la lista de datos correlacionados en un DataFrame
  df_correlated = pd.DataFrame(correlated_data)

  # Retornar el nuevo DataFrame con los tiempos de ECG y BVP correlacionados
  return df_correlated


def correlaciones_faltantes(df):
  # Calcular la media de los valores no negativos
  mean_value = df[df['PAT'] >= 0]['PAT'].mean()

  # Reemplazar los valores negativos con la media calculada
  df['PAT'] = df['PAT'].apply(lambda x: mean_value if x < 0 else x)
  return df

def parametros_PAT(df_2):
   
  # Calcular el promedio de la columna 'PAT'
  PAT_mean = df_2['PAT'].mean()

  # Calcular la desviación estándar de la columna 'PAT'
  desviacion_pat = df_2['PAT'].std()

  # Calcular la diferencia entre valores consecutivos de la columna 'PAT'
  diferencias = df_2['PAT'].diff().to_numpy()

  # Aplicar el valor absoluto a las diferencias
  diferencias_abs = np.abs(diferencias)

  # Filtrar los valores NaN
  diferencias_abs_sin_nan = diferencias_abs[~np.isnan(diferencias_abs)]

  # Calcular la desviación estándar de PATV
  # Calcular la desviación estándar y la varianza
  desviacion_estandar = np.std(diferencias_abs_sin_nan)
  varianza = np.sqrt(np.mean(np.square(diferencias_abs_sin_nan)))

  return PAT_mean, desviacion_pat, desviacion_estandar, varianza

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

#Implementacion del codigo--------------------------------------------------------------------------------------------------

# Lectura de los datos----------------------------------------------------------------------
ruta_archivo = "C:/Users/estalin.ulco/Downloads/datos_TIC/TIC/datos_entrenamiento/Student_6_Stressor.txt"
df=lectura_datos(ruta_archivo)
df=convertir_columnas_a_enteros(df)
df, tiempo_maximo_data=convertir_muestras_a_tiempo(df)

#Normalizacion de los datos-------------------------------------------------------------------------------------------------
# Implementacion de la normalizacion
df_procesada_1=normalizar_datos(df)
#print(df_procesada_1)

#Filtrado de los datos----------------------------------------------------------------------

# parametros de los filtros
# df -> dataframe con datos a trabajar
# order -> orden de transicion, valores altos -> se atenuan con mas rapidez las frecuencias 
# cutoff_high -> frecuencia de corte del filtro pasa alto
# cutoff_low -> frecuencia de corte del filtro pasa bajo
order = 5
cutoff_high = 0.7  # frecuencia de corte, ajusta este valor según sea necesario
cutoff_low = 30  # frecuencia de corte del filtro pasa bajo

df_procesada_0_ECG = senial_ECG_pam_tompkins(df_procesada_1, order, cutoff_high, cutoff_low)
df_procesada_0_BVP= senial_BVP_paso_bajo(df_procesada_1, order ,cutoff_low)

#Union de los dataframes normalizados y limpiados
df_procesada_0=pd.merge(df_procesada_0_BVP, df_procesada_0_ECG, on='Time (s)')
# print(df_procesada_0)

#Rangos de tiempo para hacer las graficas -----------------------------------------------------------------------------------

#Tiempo maximo de graficacion varia segun el dataset que se cargue Time (s)
print(f"Rango de tiempo de la toma de muestras (segundos): {tiempo_maximo_data}")


# CALCULO DE PARAMTROS - CREACION DE DATAFRAMES CON DATOS--------------------------------------------------------------------

#Picos señales ECG y BVP-----------------------------------------------------------------------------------------------------
#Ingresar el estado de estres 0 para no estresado y 1 para estresado, dependiendo del dataset:
est_estres = 1

#Intervalo de tiempo en el cual se requier realizar los calculos de los parametros RR, IBI y PAT----------------------------
tiempo_inicial = 0
tiempo_final = tiempo_maximo_data

#Configuracion de paramaetros para detectar PICOS ECG:
prominence_ECG=0.01
height_ECG=0.25
distance_ECG=250

#Configuracion de paramaetros para detectar PICOS BVP:
prominence_BVP=0.01
height_BVP=0.2
distance_BVP=200

#ECG calculo de parametros-----------------------------------------------------------------------------------
# Llamar a la función y obtener los valores devueltos
heart_rate_ECG, RR_sdnn, RR_rmssd, RR_sdsd, RR_pnn50, picos_ecg = parametros_RR(df_procesada_0,tiempo_inicial, tiempo_final,prominence_ECG,height_ECG,distance_ECG)
# Crear una matriz numpy con los valores
RR_matrix = np.array([heart_rate_ECG, RR_sdnn, RR_rmssd, RR_sdsd, RR_pnn50])

#BVP calculo de parametros----------------------------------------------------------------------------------
# Llamar a la función y obtener los valores devueltos
heart_rate_BVP, IBI_sdnn, IBI_rmssd, IBI_sdsd, IBI_pnn50, picos_bvp = parametros_IBI(df_procesada_0,tiempo_inicial, tiempo_final,prominence_BVP,height_BVP,distance_BVP)
# Crear una matriz numpy con los valores
BVP_matrix = np.array([heart_rate_BVP, IBI_sdnn, IBI_rmssd, IBI_sdsd, IBI_pnn50])

#PAT calculo de parametros----------------------------------------------------------------------------------
#Relacion de los picos ECG con sus respectivos picos BVP
# Llamada a la función para obtener el nuevo DataFrame con los tiempos de ECG y BVP correlacionados
df_correlated = RR_Correlated_intervals(picos_ecg, picos_bvp)
# print(df_correlated)

#Calcular el PAT entre ECG y BVP
# Crear un DataFrame para 'PAT' que almacena la diferencia entre 'BVP_PAT' y 'ECG_PAT'
df_PAT = pd.DataFrame()
df_PAT['PAT'] = df_correlated['BVP_PAT'] - df_correlated['ECG_PAT']
df_PAT=correlaciones_faltantes(df_PAT)

# Llamar a la función y obtener los valores devueltos
PAT_mean, desviacion_pat, desviacion_estandar, varianza = parametros_PAT(df_PAT)
# Crear una matriz numpy con los valores
PAT_matrix = np.array([PAT_mean, desviacion_pat, desviacion_estandar, varianza])
print(PAT_matrix)


Rango de tiempo de la toma de muestras (segundos): 303.449


In [16]:
import pandas as pd

#DataFrame en el cual almacenar los datos de los parametros calculados---------------------------------------------------------
# Crear un DataFrame vacío con columnas "Nombre" y "Apellido"
df_final = pd.DataFrame(columns=["Estres", "ECG_HR", "ECG_SDNN", "ECG_RMSSD", "ECG_SDSD", "ECG_PNN50", "BVP_HR", "BVP_SDNN", "BVP_RMSSD", "BVP_SDSD", "BVP_PNN50", "PAT_MEAN", "PAT_SD", "PAT_SD_PATV", "PAT_RMS_PATV"])

In [17]:
# Crear una nueva fila de datos
new_row = pd.DataFrame({
    "Estres": [est_estres],  # Asignar un valor a la columna "Estres"
    "ECG_HR": [RR_matrix[0]],
    "ECG_SDNN": [RR_matrix[1]],
    "ECG_RMSSD": [RR_matrix[2]],
    "ECG_SDSD": [RR_matrix[3]],
    "ECG_PNN50": [RR_matrix[4]],
    "BVP_HR": [BVP_matrix[0]],
    "BVP_SDNN": [BVP_matrix[1]],
    "BVP_RMSSD": [BVP_matrix[2]],
    "BVP_SDSD": [BVP_matrix[3]],
    "BVP_PNN50": [BVP_matrix[4]],
    "PAT_MEAN": [PAT_matrix[0]],
    "PAT_SD": [PAT_matrix[1]],
    "PAT_SD_PATV": [PAT_matrix[2]],
    "PAT_RMS_PATV": [PAT_matrix[3]]
})

# Agregar la nueva fila al DataFrame usando pd.concat
df_final = pd.concat([df_final, new_row], ignore_index=True)

print(df_final)

  Estres     ECG_HR   ECG_SDNN  ECG_RMSSD   ECG_SDSD  ECG_PNN50     BVP_HR  \
0      1  67.790795  82.641367  81.987708  51.092806  54.736842  67.435869   

    BVP_SDNN   BVP_RMSSD    BVP_SDSD  BVP_PNN50    PAT_MEAN     PAT_SD  \
0  109.29433  143.191246  100.095778  60.846561  487.348285  60.970713   

   PAT_SD_PATV  PAT_RMS_PATV  
0    52.991779     64.058088  


C:\Users\estalin.ulco\AppData\Local\Temp\ipykernel_11944\3305710447.py:21: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [29]:
# Guardar el DataFrame en un archivo Excel
df_final.to_excel('Datos_completos_estadísticos_estalin_ulco.xlsx', index=False, engine='openpyxl')

print("DataFrame guardado en 'df_final.xlsx'")

DataFrame guardado en 'df_final.xlsx'
